<a href="https://colab.research.google.com/github/aman9213/DL_assignment3/blob/main/seq2seq_without_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import os
import glob
from torch.utils.data import DataLoader,Dataset 
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
path = '/content/drive/MyDrive/hin'
csv_files = glob.glob(os.path.join(path, "*.csv"))
print(csv_files)
test_path=csv_files[0]
train_path=csv_files[2]
valid_path=csv_files[1]
train_data=pd.read_csv(train_path)
valid_data=pd.read_csv(valid_path)
test_data=pd.read_csv(test_path)
display((valid_data))
train_data_matrix=train_data.to_numpy()
valid_data_matrix=valid_data.to_numpy()
test_data_matrix=test_data.to_numpy()
# print(train_data_matrix[:,1])
print(valid_data_matrix.shape)

['/content/drive/MyDrive/hin/hin_test.csv', '/content/drive/MyDrive/hin/hin_valid.csv', '/content/drive/MyDrive/hin/hin_train.csv']


,jaisawal,जयसवाल
0,bajai,बजाई
1,sanghthan,संघठन
2,haiwaan,हैवान
3,nilgiri,नीलगिरि
4,drutgrami,द्रुतग्रामी
...,...,...
4090,paranshu,परांशु
4091,romanchit,रोमांचित
4092,ekamreshwar,एकाम्रेश्वर
4093,bluetooth,ब्ल्यूटूथ


(4095, 2)


In [ ]:
def tokenize(source,target,path):
    # Load the CSV data using pandas

    # data = pd.read_csv('data.csv')
    data=pd.read_csv(path)
    # Convert the data to lowercase
    data[source] = data[source].str.lower()

    # Clean and normalize Hindi text
    def clean_text(text):
        # Remove punctuations and digits
        text = re.sub(r'[^\u0900-\u097F\s]', '', text)
        text = re.sub(r'[\d]', '', text)

        # Normalize text
        text = unicodedata.normalize('NFD', text)
        text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
        text = unicodedata.normalize('NFC', text)

        return text

    data[target] = data[target].apply(clean_text)
    # print(train_data)
    # Tokenize the data
    source_chars = set()
    target_chars = set()

    for src, targ in zip(data[source],data[target]):
        source_chars.update(src)
        target_chars.update(targ)

    source_chars = sorted(list(source_chars))
    target_chars = sorted(list(target_chars))
    # print(target_chars,source_chars )
    source_char_to_id = {char: i for i, char in enumerate(source_chars)}
    target_char_to_id = {char: i for i, char in enumerate(target_chars)}
    source_id_to_char={i:char for i,char in enumerate(target_chars)}
    target_id_to_char={i:char for i,char in enumerate(target_chars)}
#     print(source_char_to_id,target_char_to_id)
    # Add padding, start-of-sequence and end-of-sequence tokens
    source_char_to_id['<PAD>'] = len(source_char_to_id)
    target_char_to_id['<PAD>'] = len(target_char_to_id)

    source_char_to_id['<SOS>'] = len(source_char_to_id)
    target_char_to_id['<SOS>'] = len(target_char_to_id)

    source_char_to_id['<EOS>'] = len(source_char_to_id)
    target_char_to_id['<EOS>'] = len(target_char_to_id)

    # Convert characters to numerical values
    data_numerical = []


    for src, targ in zip(data[source], data[target]):
        
        source_numerical = [source_char_to_id[char] for char in src]
        target_numerical = [target_char_to_id[char] for char in targ]
        source_numerical=[source_char_to_id['<SOS>']]+source_numerical+[source_char_to_id['<EOS>']]+[source_char_to_id['<PAD>']]*(28-len(src)-2)
        target_numerical=[target_char_to_id['<SOS>']]+target_numerical+[target_char_to_id['<EOS>']]+[target_char_to_id['<PAD>']]*(20-len(targ)-2)
        
        data_numerical.append([source_numerical, target_numerical])
    return data_numerical,target_id_to_char,source_id_to_char

In [ ]:
train_data_numerical,_,_=tokenize('shastragaar','शस्त्रागार',train_path)
valid_data_numerical,_,_=tokenize('jaisawal','जयसवाल',valid_path)
test_data_numerical,target_id_to_char,source_id_to_char=tokenize('thermax','थरमैक्स',test_path)


In [ ]:
#  test_data_numerical=tokenize('thermax','थरमैक्स',test_path)
class AksharantarDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
    
        return len(self.data)

    def __getitem__(self, index):
        
        source, target = self.data[index]
        source_tensor = torch.tensor(source, dtype=torch.long)
        target_tensor = torch.tensor(target, dtype=torch.long)
#         print('--->insidedataloader ',index,source_tensor.shape,target_tensor.shape)
        
        return source_tensor, target_tensor


In [ ]:
train_set=AksharantarDataset(train_data_numerical)
valid_set=AksharantarDataset(valid_data_numerical)
test_set=AksharantarDataset(test_data_numerical)

In [ ]:
train_data_set=DataLoader(train_set, batch_size=64, shuffle=True)
valid_data_set=DataLoader(valid_set, batch_size=64, shuffle=False)
test_data_set=DataLoader(test_set, batch_size=64, shuffle=False)
print(len(train_data_set))
print(len(valid_data_set))
print(len(test_data_set))

800
64
64


In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,num_layer,dropout,input_vocab_size,bidirection,cell_type):
    super(Encoder,self).__init__()
    self.input_size = input_size
    self.cell_type=cell_type
    #self.embedding_size = embeddings_size
    self.bidirection=bidirection
    self.hidden_size=hidden_size
    self.num_layer=num_layer
    self.dropout=nn.Dropout(dropout)
    self.embedding=nn.Embedding(input_vocab_size,embedding_size)
    if cell_type=='LSTM':
      if num_layer!=1: 
        self.rnn=nn.LSTM(embedding_size,hidden_size,num_layer,dropout=dropout,bidirectional=bidirection)
      else:
        self.rnn=nn.LSTM(embedding_size,hidden_size,num_layer,bidirectional=bidirection)
    elif cell_type=='GRU':
      if num_layer!=1:
        self.rnn=nn.GRU(embedding_size,hidden_size,num_layer,dropout=dropout,bidirectional=bidirection)
      else:
        self.rnn=nn.GRU(embedding_size,hidden_size,num_layer,bidirectional=bidirection)

    elif cell_type=='RNN':
      if num_layer!=1:
        self.rnn=nn.RNN(embedding_size,hidden_size,num_layer,dropout=dropout,bidirectional=bidirection)
      else:
        self.rnn=nn.RNN(embedding_size,hidden_size,num_layer,bidirectional=bidirection)



  def forward(self,inputs):
      
    batch_size=inputs.shape[0]

    if self.bidirection:
      h0=torch.randn(2*self.num_layer,batch_size,self.hidden_size).to(device)
      if self.cell_type=='LSTM':
        c0=torch.randn(2*self.num_layer,batch_size,self.hidden_size).to(device)
    else:
      h0=torch.randn(self.num_layer,batch_size,self.hidden_size).to(device)
      if self.cell_type=='LSTM':
        c0=torch.randn(self.num_layer,batch_size,self.hidden_size).to(device)
    embedding=self.dropout(self.embedding(inputs)) ### (batch,28,embedding_size)
    embedding=torch.permute(embedding,(1,0,2))     ##(28,batch,embedding)
    if self.cell_type=='LSTM':
      output,(hidden,cell)=self.rnn(embedding,(h0,c0))
      return hidden, cell
    else:
      output,hidden=self.rnn(embedding,h0)
      cell=None
      return hidden,cell
    # print("CC",output.shape,hidden.shape)

class Decoder(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,output_vocab_size,num_layer,dropout,bidirection,cell_type):
    super(Decoder,self).__init__()
    self.hidden_size=hidden_size
    self.cell_type=cell_type
    self.num_layer=num_layer
    self.dropout=nn.Dropout(dropout)
    self.embedding=nn.Embedding(output_vocab_size+3,embedding_size)       ## +3 because there are total 54+3=57 unique token
    if cell_type=='LSTM':
      if num_layer!=1:
        self.rnn=nn.LSTM(embedding_size,hidden_size,num_layer,dropout=dropout,bidirectional=bidirection)
      else:
        self.rnn=nn.LSTM(embedding_size,hidden_size,num_layer,bidirectional=bidirection)

    elif cell_type=='GRU':
      if num_layer!=1:
        self.rnn=nn.GRU(embedding_size,hidden_size,num_layer,dropout=dropout,bidirectional=bidirection)
      else:
        self.rnn=nn.GRU(embedding_size,hidden_size,num_layer,bidirectional=bidirection)
    elif cell_type=='RNN':
      if num_layer!=1:
        self.rnn=nn.RNN(embedding_size,hidden_size,num_layer,dropout=dropout,bidirectional=bidirection)
      else:
        self.rnn=nn.RNN(embedding_size,hidden_size,num_layer,bidirectional=bidirection)
        
    if bidirection:
      self.fc=nn.Linear(hidden_size*2,output_vocab_size+3)#----------------------------------------------------
    else:
      self.fc=nn.Linear(hidden_size,output_vocab_size+3)
  def forward(self,x,hidden,cell):

    x=x.unsqueeze(0)  
    embedding=self.dropout(self.embedding(x)) 
    if self.cell_type=='LSTM':  
      output,(hidden,cell)=self.rnn(embedding,(hidden,cell)) 
    else:
      output,hidden=self.rnn(embedding,hidden)
      cell=None
    prediction=self.fc(output)
    prediction=F.log_softmax(prediction,dim=1)
    prediction=prediction.squeeze(0)
    
    return prediction,hidden,cell
    
      
        
    
class seq2seq(nn.Module):
  def __init__(self,encoder,decoder):
    super(seq2seq,self).__init__()
    self.encoder=encoder
    self.decoder=decoder
      
  def forward(self,source,target,teacher_forceing=0.5):
    batch_size=source.shape[0]#64

    target_len=target.shape[1]#20
    
    target_vocab_size=54
    outputs=torch.zeros(target_len,batch_size,target_vocab_size+3).to(device)#--------------------------

    hidden,cell=self.encoder(source)

    x=target[:,0] ## for grabing <sos> token  

    
    for t in range(0,target_len):
      pred,hidden,cell=self.decoder(x,hidden,cell)  ##or there error
#             print(pred.shape) 64x57
#             print(hidden.shape) 4x64x256
      outputs[t]=pred
      best_guess=pred.argmax(1)
#             print("best",best_guess.shape) 64 batch of each time step
      x=target[:,t] if random.random()<teacher_forceing else best_guess
#             print('D',target[:,t])
    return outputs

    
        
    
    

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=6ab81ee7894ec57809c2cf0fe820d23faa282a9aa4f1f66dd48dd222480a6d1c
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_configuration={'name':'EE22s037','method':'bayes',
                     'metric':{'name':'val_acc','goal':'maximize'},
                     'parameters':{'embedding_size':{'values':[64,128,256]},
                                  'num_encoder_layer':{'values':[1,2,3]},
#                                   'num_decoder_layer':{'values':[1,2,3]},
                                  'hidden_layer_size':{'values':[64,256,512]},
                                  'cell_type':{'values':['LSTM','GRU','RNN']},
                                  'dropout':{'values':[0.2,0.3]},
                                  'bidirection':{'values':[True,False]}
                         
                                 }
    
                    }


def train():

  wandb.init()
  epochs=20
  learning_rate=0.001
  batch_size=64
  
  load_model=False
  input_size_encoder=28
  input_size_decoder=20
  input_vocab_size=29
  output_vocab_size=54
  cell_type=wandb.config.cell_type
  encoder_embedding_size=wandb.config.embedding_size
  decoder_embedding_size=wandb.config.embedding_size
  hidden_size=wandb.config.hidden_layer_size
  num_layers=wandb.config.num_encoder_layer
  enc_dropout=wandb.config.dropout
  dec_dropout=wandb.config.dropout
  bidirection=wandb.config.bidirection
  run_name="ees_{}_des_{}_hs_{}_nl_{}_ed_{}_dd_{}_ct_{}_bd_{}".format(encoder_embedding_size,decoder_embedding_size,hidden_size,num_layers,enc_dropout,dec_dropout,cell_type,bidirection)
  print("run_name:",run_name)
  encoder_net=Encoder(input_size_encoder,encoder_embedding_size,
                  hidden_size,num_layers,enc_dropout,input_vocab_size,bidirection,cell_type).to(device)
  decoder_net=Decoder(input_size_decoder,decoder_embedding_size,
                  hidden_size,output_vocab_size,num_layers,dec_dropout,bidirection,cell_type).to(device)

  model=seq2seq(encoder_net,decoder_net).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)   
  criterion=nn.CrossEntropyLoss(ignore_index=output_vocab_size)
  best_accuracy=0
  train_Loss=[]
  train_Acc=[]
  val_Loss=[]
  val_Acc=[]
  for epoch in range(epochs):

  
    model.train()
    train_accuracy=0.0
    train_loss = 0.0
    for i, (inputs, target) in enumerate(train_data_set):
        
      inputs=inputs.to(device)
      target=target.to(device)

      optimizer.zero_grad()

      output = model(inputs,target)

      output1=output[:].reshape(-1,output.shape[2])

      target1=target[:].reshape(-1)

      loss = criterion(output1, target1)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      predictions=torch.argmax(output,dim=2)#20x64
      predictions=torch.permute(predictions,(1,0))#64x20

      train_accuracy += (predictions == target).sum().item()
      
    train_accuracy= (train_accuracy/(len(train_data_set)*batch_size))*100 
    train_loss=(train_loss/(len(train_data_set)*batch_size))*100
    
    train_Acc.append(train_accuracy)
    train_Loss.append(train_loss)


    # print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, epochs, running_loss/(len(train_data_set)*batch_size)))

    model.eval()
    val_loss = 0.0
    val_accuracy=0.0
    with torch.no_grad():
      for i, (inputs, target) in enumerate(valid_data_set):
        inputs=inputs.to(device)

        target=target.to(device)

        output = model(inputs,target)#20x65x57

        output1=output[:].reshape(-1,output.shape[2])

        target1=target[:].reshape(-1)
        loss = criterion(output1, target1)
        val_loss += loss.item()
        predictions=torch.argmax(output,dim=2)#20x64
        predictions=torch.permute(predictions,(1,0))#64x20
        val_accuracy += (predictions == target).sum().item()
      val_loss=(train_loss/(len(valid_data_set)*batch_size))*100
      val_accuracy=(val_accuracy/(len(valid_data_set)*batch_size))*100
      val_Acc.append(val_accuracy)
      val_Loss.append(val_loss)
    wandb.log({#'epoch':e,
            'train_acc':train_accuracy,
            'train_loss':train_loss,
            'val_acc': val_accuracy,
            'val_loss':val_loss       
          })

  print("max__train_accuracy:",np.max(train_Acc))
  print("max__validation_accuracy:",np.max(val_Acc))
  print("min_train_loss:",np.min(train_Loss))
  print("min_validation_loss:",np.min(val_Loss))
  plt.plot(train_Loss, 'r', label="Training loss")
  plt.plot(val_Loss, 'lime', label="Validation loss")
  plt.title("Training and Validation Loss vs Number of Epochs", size=15)
  plt.xlabel("Number of epochs", size=15)
  plt.ylabel("Loss", size=15)
  plt.legend()
  plt.show()


  plt.plot(train_Acc, 'r', label="Training Accuracy")
  plt.plot(val_Acc ,'lime', label="Validation Accuracy")
  plt.title("Training and Validation Accuracy vs Number of Epochs", size=15)
  plt.xlabel("Number of epochs", size=15)
  plt.ylabel("Accuracy", size=15)
  plt.legend()
  plt.show()   
  wandb.run.name = run_name
  wandb.run.save()
  wandb.run.finish()

sweep_id=wandb.sweep(sweep=sweep_configuration,entity="amanvb-9213",project='DL-assignment3')
# sweep_id=
wandb.agent(sweep_id,function=train,count=10)
# wandb.agent(sweep_id,function=train,entity="amanvb-9213",project='DL-assignment3')




Error in callback <function _WandbInit._resume_backend at 0x7fa69cc9f490> (for pre_run_cell):


BrokenPipeError: ignored

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


1.2990672555752099
Create sweep with ID: voi0xux5
Sweep URL: https://wandb.ai/amanvb-9213/DL-assignment3/sweeps/voi0xux5
1.3056094450876117
1.3121213312260807
1.3186528077349067
1.3251735824160278


wandb: Agent Starting Run: oca5qcu6 with config:
wandb: 	bidirection: True
wandb: 	cell_type: RNN
wandb: 	dropout: 0.3
wandb: 	embedding_size: 128
wandb: 	hidden_layer_size: 64
wandb: 	num_encoder_layer: 2


1.3318048263899982
1.338411906734109
1.3448735331185162
1.3514222456142306


run_name: ees_128_des_128_hs_64_nl_2_ed_0.3_dd_0.3_ct_RNN_bd_True
1.357915315311402
1.3644536682404578
1.3709830143488944
1.377476894762367
1.3839150303974748
1.390537646599114
1.3969672247767448
1.403435481712222
1.4101556884124875
1.41668031970039
1.4232968483120203
1.4298033695667982
1.4364006156101823
1.4429384442046285
1.449508820194751
1.4561517611145973
1.462649688590318
1.469211794435978
1.4757349155843258
1.4822935056872666
1.4890103600919247
1.4955379110760987
1.5019375258125365
1.5084533621557057
1.5148312808014452
1.521337693091482
1.5278397416695952
1.5342843574471772
1.5410463442094624
1.5477069756016135
1.5542732561007142
1.5608623083680868
1.567343114875257
1.5737515478394926
1.5802827482111752
1.5867859022691846
1.5934728188440204
1.5998847908340397
1.6063380893319843
1.612957798875868
1.619281039107591
1.6258036023937166
1.6324394759722054
1.639092490077019
1.6456705438904464
1.652434594463557
1.6592104672454298
1.6659289947710931
1.6726324702613056
1.6793155418708918

wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7fa69cc9fa30> (for post_run_cell):


BrokenPipeError: ignored

In [ ]:
cn82ef7k

NameError: ignored

In [ ]:
def train_best_model():

  wandb.init()
  epochs=20
  learning_rate=0.001
  batch_size=64
  
  load_model=False
  input_size_encoder=28
  input_size_decoder=20
  input_vocab_size=29
  output_vocab_size=54
  cell_type=wandb.config.cell_type
  encoder_embedding_size=wandb.config.embedding_size
  decoder_embedding_size=wandb.config.embedding_size
  hidden_size=wandb.config.hidden_layer_size
  num_layers=wandb.config.num_encoder_layer
  enc_dropout=wandb.config.dropout
  dec_dropout=wandb.config.dropout
  bidirection=wandb.config.bidirection
  run_name="ees_{}_des_{}_hs_{}_nl_{}_ed_{}_dd_{}_ct_{}_bd_{}".format(encoder_embedding_size,decoder_embedding_size,hidden_size,num_layers,enc_dropout,dec_dropout,cell_type,bidirection)
  print("run_name:",run_name)
  encoder_net=Encoder(input_size_encoder,encoder_embedding_size,
                  hidden_size,num_layers,enc_dropout,input_vocab_size,bidirection,cell_type).to(device)
  decoder_net=Decoder(input_size_decoder,decoder_embedding_size,
                  hidden_size,output_vocab_size,num_layers,dec_dropout,bidirection,cell_type).to(device)

  model=seq2seq(encoder_net,decoder_net).to(device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)   
  criterion=nn.CrossEntropyLoss(ignore_index=output_vocab_size)
  best_accuracy=0
  train_Loss=[]
  train_Acc=[]
  val_Loss=[]
  val_Acc=[]
  for epoch in range(epochs):

  
    model.train()
    train_accuracy=0.0
    train_loss = 0.0
    for i, (inputs, target) in enumerate(train_data_set):
        
      inputs=inputs.to(device)
      target=target.to(device)

      optimizer.zero_grad()

      output = model(inputs,target)

      output1=output[:].reshape(-1,output.shape[2])

      target1=target[:].reshape(-1)

      loss = criterion(output1, target1)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      predictions=torch.argmax(output,dim=2)#20x64
      predictions=torch.permute(predictions,(1,0))#64x20

      train_accuracy += (predictions == target).sum().item()
      
    train_accuracy= (train_accuracy/(len(train_data_set)*batch_size))*100 
    train_loss=(train_loss/(len(train_data_set)*batch_size))*100
    
    train_Acc.append(train_accuracy)
    train_Loss.append(train_loss)


    # print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, epochs, running_loss/(len(train_data_set)*batch_size)))

    model.eval()
    val_loss = 0.0
    val_accuracy=0.0
    with torch.no_grad():
      for i, (inputs, target) in enumerate(valid_data_set):
        inputs=inputs.to(device)

        target=target.to(device)

        output = model(inputs,target)#20x65x57

        output1=output[:].reshape(-1,output.shape[2])

        target1=target[:].reshape(-1)
        loss = criterion(output1, target1)
        val_loss += loss.item()
        predictions=torch.argmax(output,dim=2)#20x64
        predictions=torch.permute(predictions,(1,0))#64x20
        val_accuracy += (predictions == target).sum().item()
      val_loss=(train_loss/(len(valid_data_set)*batch_size))*100
      val_accuracy=(val_accuracy/(len(valid_data_set)*batch_size))*100
      val_Acc.append(val_accuracy)
      val_Loss.append(val_loss)
      if val_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        model_wts=copy.deepcopy(model.state_dict())
        best_accuracy=val_accuracy

  print("max__train_accuracy:",np.max(train_Acc))
  print("max__validation_accuracy:",np.max(val_Acc))
  print("min_train_loss:",np.min(train_Loss))
  print("min_validation_loss:",np.min(val_Loss))
  plt.plot(train_Loss, 'r', label="Training loss")
  plt.plot(val_Loss, 'lime', label="Validation loss")
  plt.title("Training and Validation Loss vs Number of Epochs", size=15)
  plt.xlabel("Number of epochs", size=15)
  plt.ylabel("Loss", size=15)
  plt.legend()
  plt.show()


  plt.plot(train_Acc, 'r', label="Training Accuracy")
  plt.plot(val_Acc ,'lime', label="Validation Accuracy")
  plt.title("Training and Validation Accuracy vs Number of Epochs", size=15)
  plt.xlabel("Number of epochs", size=15)
  plt.ylabel("Accuracy", size=15)
  plt.legend()
  plt.show() 
  model.load_state_dict(model_wts)
  return model  

In [ ]:
best_model=train_best_model()